### 2-1) Chuẩn bị dữ liệu
 Tải xuống bộ dữ liệu News Aggregator Data Set, tạo ra dữ liệu huấn luyện (train.txt), dữ liệu
 kiểm chứng (valid.txt) và dữ liệu đánh giá (test.txt) theo hướng dẫn dưới đây.
 1. Giải nén file zip đã tải xuống, đọc hướng dẫn của file readme.txt
 2. Trích xuất ra các example (bài báo) của các báo "Reuters", "Huffington Post",
 "Businessweek", "Contactmusic.com", "Daily Mail".
 3. Sắp xếp lại các example đã trích xuất theo thứ tự ngẫu nhiên.
 4. Phân chia các example đã trích xuất với tỉ lệ 80% cho tập train, còn lại dùng 10%
 cho tập kiểm chứng và 10% cho tập đánh giá và lưu thành các file train.txt, valid.txt,
 test.txt. Trong các file, mỗi dòng lưu một example, tên của `category` và `title` của các
 bài báo được phân cách bởi dấu tab. Các file này sau này sẽ được dùng lại trong
 các bài tập tiếp theo.

Link dataset: https://archive.ics.uci.edu/dataset/359/news+aggregator

**Nội dung (CONTENT)**: 
   - **FILENAME #1**: `newsCorpora.csv` chứa thông tin về các trang tin tức với các trường như ID, tiêu đề, URL, nhà xuất bản, loại tin tức, câu chuyện, tên miền và thời gian xuất bản.
   FORMAT: `ID 	 TITLE 	 URL 	 PUBLISHER 	 CATEGORY 	 STORY 	 HOSTNAME 	 TIMESTAMP`

   - **FILENAME #2**: `2pageSessions.csv` chứa thông tin về các phiên duyệt web 2 trang với các trường như câu chuyện, tên miền, loại tin tức và URL.
   FORMAT: `STORY 	 HOSTNAME 	 CATEGORY 	 URL`

   +, CATEGORY: (b = business, t = science and technology, e = entertainment, m = health)

In [2]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
import random

# Đọc dữ liệu từ file CSV
data = pd.read_csv('newsCorpora.csv', sep='\t', header=None, names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

# Trích xuất các bài báo từ các nhà xuất bản cụ thể
publishers = ["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"]
extracted_examples = data[data['PUBLISHER'].isin(publishers)][['CATEGORY', 'TITLE']] 
# `extracted_examples` sẽ chứa một DataFrame với các cột `CATEGORY` và `TITLE`, chỉ bao gồm các bài báo từ các nhà xuất bản "Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", và "Daily Mail".

# Sắp xếp lại các example theo thứ tự ngẫu nhiên
extracted_examples = extracted_examples.sample(frac=1).reset_index(drop=True)

# Phân chia dữ liệu
train_size = int(0.8 * len(extracted_examples))
valid_size = int(0.1 * len(extracted_examples))

train_data = extracted_examples[:train_size]
valid_data = extracted_examples[train_size:train_size + valid_size]
test_data = extracted_examples[train_size + valid_size:]

# Lưu vào các file
train_data.to_csv('train.txt', sep='\t', header=False, index=False)
valid_data.to_csv('valid.txt', sep='\t', header=False, index=False)
test_data.to_csv('test.txt', sep='\t', header=False, index=False)


   ```python
   extracted_examples = data[data['PUBLISHER'].isin(publishers)][['CATEGORY', 'TITLE']]
   ```
   - `data['PUBLISHER'].isin(publishers)`: Phần này kiểm tra từng giá trị trong cột `PUBLISHER` của DataFrame `data` để xem nó có nằm trong danh sách `publishers` hay không. Kết quả là một Series boolean (True/False) cho biết mỗi hàng có thuộc về một trong các nhà xuất bản trong danh sách hay không.
   - `data[...]`: Sử dụng Series boolean này để lọc DataFrame `data`, chỉ giữ lại các hàng mà nhà xuất bản nằm trong danh sách `publishers`.
   - `[['CATEGORY', 'TITLE']]`: Sau khi lọc, chỉ giữ lại hai cột `CATEGORY` và `TITLE` từ DataFrame đã lọc. Kết quả cuối cùng là một DataFrame mới `extracted_examples` chỉ chứa các bài báo từ các nhà xuất bản đã chỉ định, với thông tin về danh mục và tiêu đề của bài báo.



### 2-2) Khám phá dữ liệu
 Thực hiện các công việc sau:
 ● Thốngkêsốsamples cho từng label trong dữ liệu trong tập train, valid và test
 ● Tính số lượng word trung bình, số lượng word lớn nhất, nhỏ nhất trong các samples
 trong tập train, valid, test

In [3]:
import pandas as pd

# Đọc dữ liệu từ các file
train_data = pd.read_csv('train.txt', sep='\t', header=None, names=['CATEGORY', 'TITLE'])
valid_data = pd.read_csv('valid.txt', sep='\t', header=None, names=['CATEGORY', 'TITLE'])
test_data = pd.read_csv('test.txt', sep='\t', header=None, names=['CATEGORY', 'TITLE'])

# Bước 2: Thống kê số samples cho từng label
train_counts = train_data['CATEGORY'].value_counts()
valid_counts = valid_data['CATEGORY'].value_counts()
test_counts = test_data['CATEGORY'].value_counts()

print("Số lượng samples trong tập train:")
print(train_counts)
print("\nSố lượng samples trong tập valid:")
print(valid_counts)
print("\nSố lượng samples trong tập test:")
print(test_counts)

# Bước 3: Tính số lượng từ trung bình, lớn nhất, nhỏ nhất
def word_statistics(data):
    # Tính số lượng từ trong mỗi tiêu đề
    word_counts = data['TITLE'].apply(lambda x: len(x.split()))
    return {
        'mean': word_counts.mean(),
        'max': word_counts.max(),
        'min': word_counts.min()
    }

train_stats = word_statistics(train_data)
valid_stats = word_statistics(valid_data)
test_stats = word_statistics(test_data)

print("\nThống kê số lượng từ trong tập train:")
print(f"Số lượng từ trung bình: {train_stats['mean']}")
print(f"Số lượng từ lớn nhất: {train_stats['max']}")
print(f"Số lượng từ nhỏ nhất: {train_stats['min']}")

print("\nThống kê số lượng từ trong tập valid:")
print(f"Số lượng từ trung bình: {valid_stats['mean']}")
print(f"Số lượng từ lớn nhất: {valid_stats['max']}")
print(f"Số lượng từ nhỏ nhất: {valid_stats['min']}")

print("\nThống kê số lượng từ trong tập test:")
print(f"Số lượng từ trung bình: {test_stats['mean']}")
print(f"Số lượng từ lớn nhất: {test_stats['max']}")
print(f"Số lượng từ nhỏ nhất: {test_stats['min']}")

Số lượng samples trong tập train:
CATEGORY
b    4502
e    4229
t    1213
m     728
Name: count, dtype: int64

Số lượng samples trong tập valid:
CATEGORY
b    554
e    534
t    154
m     92
Name: count, dtype: int64

Số lượng samples trong tập test:
CATEGORY
b    571
e    516
t    157
m     90
Name: count, dtype: int64

Thống kê số lượng từ trong tập train:
Số lượng từ trung bình: 10.468047226386807
Số lượng từ lớn nhất: 201
Số lượng từ nhỏ nhất: 2

Thống kê số lượng từ trong tập valid:
Số lượng từ trung bình: 10.41904047976012
Số lượng từ lớn nhất: 19
Số lượng từ nhỏ nhất: 2

Thống kê số lượng từ trong tập test:
Số lượng từ trung bình: 10.515742128935532
Số lượng từ lớn nhất: 17
Số lượng từ nhỏ nhất: 2


### 2-3) Huấn luyện mô hình phân loại
 Hãy huấn luyện mô hình phân loại văn bản trên dữ liệu train
 ● Cóthểsửdụngbất kỳ thuật toán phân loại văn bản nào mà bạn biết
 ● Trong quá trình xây dựng mô hình phân loại văn bản, hãy sử dụng tập valid để lựa
 chọn siêu tham số (hyper-parameters)

2 phương pháp lựa chọn siêu tham số: 

1. Grid Search:
   - Tìm kiếm có hệ thống qua tất cả các kết hợp tham số được xác định trước.
   - Đảm bảo tìm ra kết hợp tham số tốt nhất trong không gian tìm kiếm đã định.
   - Hiệu quả cho không gian tham số nhỏ và rời rạc.
   - Có thể rất chậm với nhiều tham số hoặc khoảng giá trị lớn.

2. Random Search:
   - Chọn ngẫu nhiên các kết hợp tham số từ phân phối xác định trước.
   - Có thể khám phá không gian tham số rộng hơn trong cùng thời gian.
   - Hiệu quả hơn cho không gian tham số lớn và liên tục.
   - Có thể bỏ qua kết hợp tham số tối ưu, nhưng thường tìm ra giải pháp gần tối ưu nhanh hơn.

=> Grid Search đảm bảo tìm ra giải pháp tốt nhất trong không gian tìm kiếm đã định, trong khi Random Search cho phép khám phá không gian rộng hơn và thường nhanh hơn, đặc biệt với nhiều tham số.

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

# Bước 1: Đọc dữ liệu từ các file
train_data = pd.read_csv('train.txt', sep='\t', header=None, names=['CATEGORY', 'TITLE'])
valid_data = pd.read_csv('valid.txt', sep='\t', header=None, names=['CATEGORY', 'TITLE'])

# Bước 2: Tiền xử lý dữ liệu
vectorizer = TfidfVectorizer(max_features=5000)  # Giới hạn số lượng đặc trưng để giảm thời gian tính toán
X_train = vectorizer.fit_transform(train_data['TITLE'])
y_train = train_data['CATEGORY']
X_valid = vectorizer.transform(valid_data['TITLE'])
y_valid = valid_data['CATEGORY']

# Bước 3: Huấn luyện mô hình SVM
svm_model = SVC(kernel='linear', probability=True)

# Bước 4: Tối ưu hóa siêu tham số
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto']
}
grid_search = GridSearchCV(svm_model, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# In ra siêu tham số tốt nhất
print("Siêu tham số tốt nhất:", grid_search.best_params_)

# Sử dụng mô hình với siêu tham số tốt nhất
best_model = grid_search.best_estimator_

# Bước 5: Đánh giá mô hình trên tập valid
y_pred = best_model.predict(X_valid)

# In ra báo cáo phân loại
print("\nBáo cáo phân loại:")
print(classification_report(y_valid, y_pred))

# In ra độ chính xác
accuracy = accuracy_score(y_valid, y_pred)
print(f"Độ chính xác trên tập valid: {accuracy:.2f}")

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Siêu tham số tốt nhất: {'C': 1, 'gamma': 'scale'}

Báo cáo phân loại:
              precision    recall  f1-score   support

           b       0.93      0.95      0.94       554
           e       0.92      0.98      0.95       534
           m       0.88      0.66      0.76        92
           t       0.85      0.73      0.78       154

    accuracy                           0.92      1334
   macro avg       0.90      0.83      0.86      1334
weighted avg       0.91      0.92      0.91      1334

Độ chính xác trên tập valid: 0.92


### 2-4) Đánh giá mô hình phân loại
 Đánh giá mô hình phân loại mà bạn đã xây dựng ở bài 2-3) trên tập dữ liệu test. Tính các
 chỉ số precision, recall, F1 cho từng nhãn và các chỉ số trung bình (macro-average)

In [6]:
from sklearn.metrics import classification_report, precision_recall_fscore_support

# Đọc dữ liệu test
test_data = pd.read_csv('test.txt', sep='\t', header=None, names=['CATEGORY', 'TITLE'])

# Tiền xử lý dữ liệu test
X_test = vectorizer.transform(test_data['TITLE'])
y_test = test_data['CATEGORY']

# Dự đoán nhãn cho tập test
y_pred = best_model.predict(X_test)

# In báo cáo phân loại
print("Báo cáo phân loại chi tiết:")
print(classification_report(y_test, y_pred))

# Tính precision, recall, F1 cho từng nhãn và macro-average
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None)
macro_precision, macro_recall, macro_f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')

# In kết quả macro-average
print("\nKết quả trung bình (macro-average):")
print(f"Macro Precision: {macro_precision:.4f}")
print(f"Macro Recall: {macro_recall:.4f}")
print(f"Macro F1-score: {macro_f1:.4f}")

Báo cáo phân loại chi tiết:
              precision    recall  f1-score   support

           b       0.91      0.95      0.93       571
           e       0.91      0.97      0.94       516
           m       0.94      0.68      0.79        90
           t       0.85      0.65      0.74       157

    accuracy                           0.91      1334
   macro avg       0.90      0.81      0.85      1334
weighted avg       0.91      0.91      0.90      1334


Kết quả trung bình (macro-average):
Macro Precision: 0.9027
Macro Recall: 0.8137
Macro F1-score: 0.8492


###  2-5) Triển khai mô hình phân loại thành API
 Hãy triển khai mô hình phân loại văn bản mà bạn đã xây dựng thành API theo đặc tả sau
 đây. Có các end-point như mô tả dưới đây.
 Base URL: http://localhost:2005
 2-5-1. Liệt kê danh sách các nhãn
 API này sẽ liệt kê danh sách các nhãn (labels) trong tập nhãn.
- End-point:
```
/list_label
 Method: GET
```
- Request body
- Request parameters: NONE

Lưu mô hình và vectorizer sau khi huấn luyện

In [7]:
import joblib  # Nhập thư viện joblib để lưu và tải mô hình

# Lưu mô hình sau khi huấn luyện mô hình
joblib.dump(best_model, 'best_model.joblib')  # Lưu mô hình đã huấn luyện vào file 'best_model.joblib'
joblib.dump(vectorizer, 'vectorizer.joblib')  # Lưu vectorizer đã sử dụng để chuyển đổi văn bản vào file 'vectorizer.joblib'

['vectorizer.joblib']

In [3]:
from flask import Flask, jsonify
import joblib

app = Flask(__name__)

# Load mô hình và vectorizer đã được huấn luyện
model = joblib.load('best_model.joblib')
vectorizer = joblib.load('vectorizer.joblib')

# Danh sách các nhãn
labels = ['b', 'e', 'm', 't']
    
@app.route('/list_label', methods=['GET'])  # Định nghĩa route cho endpoint '/list_label' với phương thức GET
def list_label():
    return jsonify({"labels": labels})  # Trả về danh sách nhãn dưới dạng JSON

from flask import cli  # Nhập cli từ Flask
cli.show_server_banner = lambda *_: None  # Tắt banner hiển thị khi chạy server

if __name__ == '__main__':  # Kiểm tra xem file có phải là file chính được chạy không
    app.run(host='localhost', port=2005, debug=True, use_reloader=False)  # Chạy ứng dụng Flask trên localhost với port 2005

 * Running on http://localhost:2005
Press CTRL+C to quit
127.0.0.1 - - [08/Oct/2024 20:56:23] "GET /list_label HTTP/1.1" 200 -
127.0.0.1 - - [08/Oct/2024 20:56:24] "GET /list_label HTTP/1.1" 200 -


![images\API_GET_list_label.png](./images/API_GET_list_label.png)


### 2-5-2. Phân loại văn bản

API này nhận đầu vào là một đoạn text và trả ra label của nhãn đó cùng với xác suất (confidence score) tương ứng.

#### Endpoint
```
/classify
```

#### Method
```
POST
```

#### Parameters

| Name | Type   | Description          |
|------|--------|----------------------|
| text | string | Nội dung text đầu vào |

#### Response

| JSON Key         | Type   | Description                     |
|------------------|--------|---------------------------------|
| text             | string | Nội dung input text            |
| predicted_label  | string | Nhãn dự đoán                   |
| prob             | float  | Xác suất của nhãn dự đoán      |



In [4]:
from flask import Flask, request, jsonify
import joblib
import numpy as np

app = Flask(__name__)

# Tải mô hình và vectorizer đã được huấn luyện
model = joblib.load('best_model.joblib')  # Tải mô hình phân loại
vectorizer = joblib.load('vectorizer.joblib')  # Tải vectorizer đã sử dụng để chuyển đổi văn bản

@app.route('/classify', methods=['POST'])  # Định nghĩa route cho endpoint '/classify' với phương thức POST
def classify():
    # Lấy dữ liệu từ request
    data = request.json  # Lấy dữ liệu JSON từ yêu cầu
    text = data.get('text', '')  # Lấy nội dung văn bản từ dữ liệu, mặc định là chuỗi rỗng

    if not text:  
        return jsonify({"error": "No text provided"}), 400  # Trả về lỗi 400 nếu không có văn bản

    # Chuyển đổi văn bản thành vector
    text_vector = vectorizer.transform([text])  # Chuyển đổi văn bản thành vector sử dụng vectorizer

    # Dự đoán nhãn và xác suất
    predicted_label = model.predict(text_vector)[0]  # Dự đoán nhãn cho vector văn bản
    probabilities = model.predict_proba(text_vector)[0]  # Lấy xác suất cho từng nhãn
    
    # Lấy xác suất cao nhất
    max_prob = np.max(probabilities)  # Tìm xác suất cao nhất trong các xác suất dự đoán

    # Chuẩn bị response
    response = {
        "text": text,  # Nội dung văn bản đầu vào
        "predicted_label": predicted_label,  # Nhãn dự đoán
        "prob": float(max_prob)  # Xác suất của nhãn dự đoán
    }

    return jsonify(response)  # Trả về phản hồi dưới dạng JSON

if __name__ == '__main__':
    app.run(host='localhost', port=2005, debug=True, use_reloader=False)  # Chạy ứng dụng Flask trên localhost với port 2005

 * Running on http://localhost:2005
Press CTRL+C to quit
127.0.0.1 - - [08/Oct/2024 20:57:23] "POST /classify HTTP/1.1" 200 -
127.0.0.1 - - [08/Oct/2024 20:57:28] "POST /classify HTTP/1.1" 200 -


TEST API: Test API bằng Postman hoặc công cụ khác.
1. Gửi dữ liệu dưới dạng JSON trong body của request, không phải query parameters.
Ví dụ: `{"text": "Australia men have third best life ex women"}`
- Và đặt header `Content-Type: application/json`.

2. Hoặc Import nhanh bằng: 
```
{
  "info": {
    "name": "News Aggregator API",
    "_postman_id": "unique-id-here",
    "description": "API for classifying news articles",
    "schema": "https://schema.getpostman.com/json/collection/v2.1.0/collection.json"
  },
  "item": [
    {
      "name": "Classify Text",
      "request": {
        "url": "http://localhost:2005/classify",
        "method": "POST",
        "header": [
          {
            "key": "Content-Type",
            "value": "application/json"
          }
        ],
        "body": {
          "mode": "raw",
          "raw": "{\n    \"text\": \"Australia men have third best life expectancy in the world whereas its women\"\n}"
        },
        "description": "Classify a given text"
      },
      "response": []
    }
  ]
}
```

![images\API_GET_classify.png](./images/API_GET_classify.png)

### API 

### 1. So sánh 4 phương thức trong API

Trong API, có bốn phương thức HTTP phổ biến mà bạn có thể sử dụng để thực hiện các thao tác khác nhau. Dưới đây là so sánh giữa chúng:

| Phương thức | Mô tả | Sử dụng |
|-------------|-------|---------|
| **GET**     | Lấy dữ liệu từ máy chủ. Không có tác động đến dữ liệu trên máy chủ. | Sử dụng để truy xuất thông tin, ví dụ: lấy danh sách các nhãn. |
| **POST**    | Gửi dữ liệu đến máy chủ để tạo mới hoặc cập nhật tài nguyên. | Sử dụng để gửi dữ liệu, ví dụ: phân loại văn bản. |
| **PUT**     | Cập nhật toàn bộ tài nguyên trên máy chủ. | Sử dụng để cập nhật một tài nguyên hiện có, ví dụ: cập nhật thông tin của một bài viết. |
| **DELETE**  | Xóa tài nguyên trên máy chủ. | Sử dụng để xóa một tài nguyên, ví dụ: xóa một bài viết. |

### 2. Các phương pháp phổ biến

Dưới đây là một số phương pháp phổ biến trong API:

- **REST (Representational State Transfer)**: 
  - Là một kiến trúc API phổ biến, sử dụng các phương thức HTTP (GET, POST, PUT, DELETE) để thực hiện các thao tác CRUD (Create, Read, Update, Delete).
  - Dữ liệu thường được trả về dưới dạng JSON hoặc XML.

- **GraphQL**:
  - Là một ngôn ngữ truy vấn cho API, cho phép người dùng yêu cầu chính xác dữ liệu mà họ cần.
  - Khác với REST, nơi mà mỗi endpoint trả về một loại dữ liệu cụ thể, GraphQL cho phép truy vấn nhiều loại dữ liệu từ một endpoint duy nhất.

- **SOAP (Simple Object Access Protocol)**:
  - Là một giao thức truyền thông dựa trên XML, thường được sử dụng trong các dịch vụ web.
  - SOAP yêu cầu một cấu trúc chặt chẽ và thường sử dụng WSDL (Web Services Description Language) để mô tả dịch vụ.

- **gRPC (Google Remote Procedure Call)**:
  - Là một framework RPC (Remote Procedure Call) được phát triển bởi Google, sử dụng HTTP/2 và Protocol Buffers.
  - gRPC cho phép giao tiếp giữa các dịch vụ với hiệu suất cao và hỗ trợ nhiều ngôn ngữ lập trình.

Mỗi phương pháp và kiến trúc API có ưu điểm và nhược điểm riêng, và việc lựa chọn phương pháp nào phụ thuộc vào yêu cầu cụ thể của dự án và môi trường phát triển.